In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd
import numpy as np

In [2]:
url='https://mediabiasfactcheck.com/filtered-search/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [3]:
#content.find(text='getData()')
scripts = soup.find_all('script')
data = scripts[34] #34 is script that includes the JSON file that we want

In [4]:
for elem in soup(text=re.compile("function getData()")):
    data = elem.split("current_json = ", 2)
    data = data[2].split("updateFilters()")
    data = data[0].split(";")
jsondata = data[0] #data[0] contains the json file we are looking for 

In [5]:
result = json.loads(json.dumps(jsondata))
df = pd.DataFrame.from_dict(eval(result), orient='index') #have to do eval(result) because in this case, result isn't the dictionary itself, it's a string representation of a dictionary – eval makes it into dictionary form 
df.head(15)

,b,d,h,L,n,r,u,c
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States
,Left,,https://,0,Vote,Mixed,https://mediabiasfactcheck.com/vote/,
www.whitehouse.gov,Right,www.whitehouse.gov,https://www.whitehouse.gov,"13,742,825",1600 Daily,Mostly Factual,https://mediabiasfactcheck.com/1600-daily/,United States
21stcenturywire.com,Conspiracy-Pseudoscience,21stcenturywire.com,https://21stcenturywire.com,"332,291",21st Century Wire,Mixed,https://mediabiasfactcheck.com/21st-century-wire/,United States
us24news.com,Questionable Sources,us24news.com,https://us24news.com,527,24 News,Low,https://mediabiasfactcheck.com/24-news/,United States
247newsmedia.com,Questionable Sources,247newsmedia.com,https://247newsmedia.com,213,247 News Media,Very Low,https://mediabiasfactcheck.com/247-news-media/,Macedonia
secondamendmentdaily.com,Questionable Sources,secondamendmentdaily.com,https://secondamendmentdaily.com,0,2nd Amendment Daily News,Low,https://mediabiasfactcheck.com/2nd-amendment-d...,United States


In [23]:
###SAVES THE JSON FILE, DON'T UNCOMMENT UNLESS YOU WANT TO DOWNLOAD A COPY OF THE "RESULTS" OBJECT TO YOUR LOCAL

#filewrite = open("MBFCjson.txt", "w")
#filewrite.write(result)

In [7]:
df = df.rename(columns={"b": "Bias", "d": "URL", "h": "Full URL", "L": "References", "n": 
                        "Name", "r": "Factualness", "u": "MBFC analysis", "c": "Country"})
df.head()

,Bias,URL,Full URL,References,Name,Factualness,MBFC analysis,Country
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States
,Left,,https://,0,Vote,Mixed,https://mediabiasfactcheck.com/vote/,


In [8]:
print (df['Bias'].unique()) #looking at all the different bias labels

['Questionable Sources' 'Least Biased' 'Left' 'Right'
 'Conspiracy-Pseudoscience' 'Right-Center' 'Left-Center' 'Satire'
 'Pro-Science']


In [9]:
#numerical value for each bias 
bias_dictionary = {"Left": -7, "Left-Center": -3.5, "Least Biased": 0, "Right-Center": 3.5, "Right": 7, 
                  "Questionable Sources": None, "Conspiracy-Pseudoscience": None, "Satire": None, "Pro-Science": None}
df['Bias Rating'] = df['Bias'].map(bias_dictionary)

In [10]:
final_data = pd.read_csv('../data/final_data.csv')
#final_data['Source'].unique()

In [11]:
#temporarily commented out code for all sources in scraped data

#for name in df['Name'].unique():
    #print(name)

In [12]:
#find all the sources that are named differently in the final data
unique_sources = final_data[final_data['Source'].isin(df['Name'].unique()) == False]['Source'].unique()
unique_sources

array(['ABC', 'AP', 'BBC', 'BuzzFeed', 'CBS', 'CNSNews', 'Counterpunch',
       'Daily Kos', 'Daily Wire', 'EPI', 'Financial Times', 'Fortune',
       'Forward', 'FreeSpeech TV', 'IJR', 'InfoWars', 'Intercept',
       'Marketwatch', 'National Public Radio', 'NewsPunch', 'NewsMax',
       'OZY', 'Patribotics', 'PBS', 'Progressive, The', 'ProPublica',
       'RedState', 'The Atlantic', 'The New Yorker', 'Time', 'UrNews24',
       'Truthout', 'Vice', 'Wall Street Journal',
       'Washington Free Beacon', 'Washington Examiner',
       'Washington Monthly', 'Washington Post', 'Weather.com',
       'World Truth TV', 'Washington Times', 'Wonkette', 'WorldNetDaily'],
      dtype=object)

In [13]:
#change those names to match (this is a temporary solution)
'''
df.replace({'Name': {'ABC News': 'ABC', 'Associated Press': 'AP', 'BBC News': 'BBC', 'Buzzfeed': 'BuzzFeed',
                    'CBS News': 'CBS',
                    'CNS News': 'CNSNews',
                    'CounterPunch': "Counterpunch",
                     '': 'Daily Kos'
                                                                        
                   
                    }}).head(40)
'''
df = df.replace({'Name': {'ABC News': unique_sources[0],
    'Associated Press': unique_sources[1], 
    'BBC News': unique_sources[2],
    'Buzzfeed': unique_sources[3],
    'CBS News': unique_sources[4],
    'CNS News': unique_sources[5],
    'CounterPunch': unique_sources[6],
    'The Daily Wire': unique_sources[8],
    'Economic Policy Institute': unique_sources[9],
    'Financal Times': unique_sources[10],
    'Fortune Magazine': unique_sources[11],
    'The Forward': unique_sources[12],
    'FSTv (Freespeech.org)': unique_sources[13],
    'Independent Journal Review': unique_sources[14],
    'Infowars': unique_sources[15],
    'The Intercept': unique_sources[16],
    'MarketWatch': unique_sources[17],
    'NPR': unique_sources[18],
    'News Punch': unique_sources[19],
    'Newsmax': unique_sources[20],
    'Ozy Media': unique_sources[21],
    'Patribotics Blog': unique_sources[22],
    'PBS News Hour': unique_sources[23],
    'The Progressive': unique_sources[24],
    'Propublica': unique_sources[25],
    'RedStateWave': unique_sources[26],
    'Atlantic': unique_sources[27],
    'New Yorker': unique_sources[28],
    'Time Magazine': unique_sources[29],
    'TruthOut': unique_sources[31],
    'Vice News': unique_sources[32],                   
}})

#wtf no washington post? no washington examiner? no washington ANYTHING? need to add suspiciously missing sources into the data

In [14]:
#ok so as it turns out, the scraped webpage doesn't have sources past "War on the Rocks" alphabetically
#got to find some way of amending that

In [15]:
name_and_bias = pd.concat([df['Name'], df['Bias Rating']], axis=1)
#rename columns for merging purposes
name_and_bias = name_and_bias.rename(columns={'Name': 'Source'})
name_and_bias.head()

,Source,Bias Rating
100percentfedup.com,100% Fed up,NaN
1010wins.radio.com,1010 WINS AM,0.0
www.10news.one,10News.one,NaN
12news.com,12 News KPNX,0.0
,Vote,-7.0


In [16]:
final_data.head(50)

,Url,Author,Date,Header,Body,n_links,Source,Bias,Quality
0,https://abcnews.go.com/Politics/abortion-right...,Devin Dwyer,"Wed, 17 Apr 2019 10:14:00 GMT",Abortion rights group asks Supreme Court to st...,Abortion rights advocates have asked the U.S. ...,3.0,ABC,1.67,49.00
1,https://abcnews.go.com/Politics/appeals-court-...,Ali Dukakis,"Tue, 26 Feb 2019 09:05:00 GMT",Appeals court says special counsel Robert Muel...,A federal appeals court rejected the most dire...,2.0,ABC,0.67,51.67
2,https://abcnews.go.com/Politics/attorney-gener...,Luke Barr,"Wed, 17 Apr 2019 14:02:00 GMT",Attorney general orders some asylum seekers to...,As part of the Trump administration's effort t...,6.0,ABC,-2.75,43.50
3,https://abcnews.go.com/Politics/donald-trump-t...,Meridith McGraw,"Tue, 19 Mar 2019 12:44:00 GMT","Donald Trump and 'the Trump of the Tropics,' B...","President Donald Trump and ""the Trump of the T...",10.0,ABC,-4.33,52.67
4,https://abcnews.go.com/Politics/electoral-coll...,Matthew Dowd,"Tue, 19 Mar 2019 21:39:00 GMT",The Electoral College limits the campaign play...,"U.S Senator Elizabeth Warren, who is competing...",5.0,ABC,-10.00,32.00
5,https://abcnews.go.com/Politics/facebook-agree...,Anne Flaherty,"Tue, 19 Mar 2019 13:38:00 GMT",Facebook agrees to hide demographics from land...,Facebook announced Tuesday that it will block ...,6.0,ABC,-2.33,52.33
6,https://abcnews.go.com/Politics/learned-2020-c...,Benjamin Siegel \nand,"Wed, 17 Apr 2019 01:41:00 GMT",What we've learned from 2020 candidates' tax r...,A handful of Democratic candidates for preside...,14.0,ABC,-0.67,47.67
7,https://abcnews.go.com/Politics/michael-cohen-...,Aaron Katersky,"Tue, 26 Feb 2019 13:34:00 GMT","Michael Cohen, President Trump’s former person...",Michael Cohen has been officially disbarred in...,3.0,ABC,3.50,56.75
8,https://abcnews.go.com/Politics/mueller-report...,Katherine Faulders,"Tue, 16 Apr 2019 13:51:00 GMT","As Mueller report release nears, White House p...",While President Donald Trump and the White Hou...,2.0,ABC,-2.33,54.33
9,https://abcnews.go.com/Politics/republican-cen...,Adam Kelsey,"Tue, 26 Feb 2019 20:10:00 GMT",Republican at center of North Carolina electio...,"Mark Harris, the Republican candidate in North...",6.0,ABC,-0.75,53.25


In [17]:
#merge the two dataframes
final_data = pd.merge(final_data, name_and_bias, on='Source', how='inner') 
#final_data

In [18]:
#final_data.loc[final_data['Bias Rating'].isna()]

In [19]:
#cleaning out data without bias rating
#final_data = final_data.dropna(subset=['Bias Rating'])

In [20]:
#REQUIRES the name "Source" for the source
#drops Na's for bias ratings (since not all have an easy rating to access)
def merge_ratings(final_data):
    final_data = pd.merge(final_data, name_and_bias, on='Source', how='inner') 
    final_data = final_data.dropna(subset=['Bias Rating'])
    return final_data